In [1]:
# Split into 5000 tick images with 1000 tick shift
import mido
from utils import mido_setup, get_track_instrument_type
import os
import pandas
from collections import Counter
from matplotlib import pyplot as plt

In [8]:
initial_path = "/Users/edlgg/repos/dissertation/midis/v3/control_change/0_vel/120/"
path = initial_path
images_path = "/Users/edlgg/repos/dissertation/images/v3"
# images_out_path = "/Users/edlgg/repos/dissertation/results/images/v1"
# midi_out_path = "/Users/edlgg/repos/dissertation/results/midis/v1"

In [9]:
mid = mido.MidiFile(path + "pink_panther2.mid")

In [10]:
def get_max_length_mid(mid):
    max_length = 0
    for track in mid.tracks:
        length = 0
        for msg in track:
            if msg.type == "note_on":
                length = length + msg.time
        if length > max_length:
            max_length = length
    return max_length
get_max_length_mid(mid)

40757

In [11]:
def track_to_image(track, length):
    image = [ [-1]*(length+1) for _ in range(90) ]
    note_locs = []
    time_step = 0
    for msg in track:
        if msg.type == "note_on":
            time_step = time_step + int(msg.time)
            image[msg.note][time_step] = msg.velocity
            if msg.velocity != 0:
                note_locs.append((msg.note, time_step))
                
    for note_loc in note_locs:
        y = note_loc[0]
        x = note_loc[1]
        note = image[y][x]
        while image[y][x] != 0:
            image[y][x] = note
            x = x + 1
            
    for i, row in enumerate(image):
        for j, e in enumerate(row):
            if image[i][j] == -1:
                image[i][j] = 0
                
    image = image[20:]
    inverted_image = []
    for row in image:
        inverted_image.insert(0, row)
    return inverted_image
# image = track_to_image(mid.tracks[0], 108595)

def test_track_to_image(initial_path):
    song = mido.MidiFile(initial_path + "pink_panther2.mid")
    le = get_max_length_mid(song)
    image = track_to_image(song.tracks[0], le)
    
    x = [row[:2000] for row in image]
    plt.figure(figsize=(100,300))
    plt.imshow(x, cmap='gray', interpolation='nearest', vmin=0, vmax=120)
    plt.show()
# test_track_to_image(initial_path)

In [12]:
image = track_to_image(mid.tracks[0], 108595)
image = [row[:2500] for row in image]
count = 0
for row in image:
    for i in row:
        if i != 0:
            count = count + 1

In [13]:
def create_image_from_mid(mid):
    length = get_max_length_mid(mid)
    guitar_image = track_to_image(mid.tracks[0], length)
    bass_image = track_to_image(mid.tracks[1], length)
    return guitar_image + bass_image

def test_create_image_from_mid(mid):
    image = create_image_from_mid(mid)
    x = [row[-2000:] for row in image]
    plt.figure(figsize=(100,300))
    plt.imshow(x, cmap='gray', interpolation='nearest', vmin=0, vmax=120)
    plt.show()
# test_create_image_from_mid(mid)

In [14]:
def split_image(image, stride):
    images = []
    x = 2000
    while x <= len(image[0]):
        sub_image = [row[x-2000:x] for row in image]
        x = x + stride
        guitar = sub_image[:70]
        count_guitar = 0
        for row in guitar:
            for e in row:
                if e != 0:
                    count_guitar = count_guitar + 1
        bass = sub_image[70:]
        count_bass = 0
        for row in guitar:
            for e in row:
                if e != 0:
                    count_bass = count_bass + 1
        if min(count_guitar, count_bass) > 200 and max(count_guitar, count_bass) > 1000:
            images.append(sub_image)
    return images

In [17]:
image = create_image_from_mid(mid)
images = split_image(image, 500)
image = images[0]

In [13]:
def create_images(initial_path, end_path):
    pass

In [14]:
def create_midi_from_image(image):
    pass

In [187]:
def save_images_to_csv(images):
    pass

In [ ]:
def load_images_from_csv(path):
    pass